This script cleans the superposition PDB (containing all the superposed
pockets) by removing the atoms that are on the core of the protein.

In [1]:
import numpy as np

In [ ]:
ref_structure = "/home/alex/Documents/pocket_tool/data/ref_gpcr/data/a2a_6gdg_opm_rotated.pdb"
superposition = "/home/alex/sshfs_mountpoints/verde/combine_pockets/output_grid_2a.pdb"

In [3]:
TM_RESIDS_REF = "1-34,39-69,73-108,117-142,173-213,219-259,266-291"

TM1_RESIDS = list(range(1, 35))
TM2_RESIDS = list(range(39, 70))
TM3_RESIDS = list(range(73, 109))
TM4_RESIDS = list(range(117, 143))
TM5_RESIDS = list(range(173, 214))
TM6_RESIDS = list(range(219, 260))
TM7_RESIDS = list(range(266, 292))

In [6]:

def read_pocket_coordinates(pdb_file):
    """
    Reads atomic coordinates from a PDB file and returns them as a list of
    (x, y, z) tuples.
    Parameters:
        pdb_file (str): Path to the PDB file to be read.
    Returns:
        List[Tuple[float, float, float]]: A list of 3D coordinates (x, y, z)
        for each atom found in the file.
    """
    with open(pdb_file, 'r') as file:
        lines = file.readlines()

    coordinates_and_betas = []
    for line in lines:
        if line.startswith("ATOM") or line.startswith("HETATM"):
            b_factor = float(line[60:66].strip())
            x = float(line[30:38].strip())
            y = float(line[38:46].strip())
            z = float(line[46:54].strip())
            coordinates_and_betas.append([[x, y, z], b_factor])

    return coordinates_and_betas


def read_ca_coordinates_per_helix(pdb_file):
    """
    Reads C-alpha coordinates for each helix from a PDB file.
    Parameters:
        pdb_file (str): Path to the PDB file to be read.
    Returns:
        Dict[str, List[Tuple[float, float, float]]]: A dictionary where keys
        are helix names and values are lists of C-alpha coordinates (x, y, z).
    """
    with open(pdb_file, 'r') as file:
        lines = file.readlines()

    ca_coordinates = {}
    for line in lines:
        line = line.strip()
        if not (line.startswith("ATOM") and line[13:15] == "CA"):
            continue

        resid = int(line[22:26].strip())
        x = float(line[30:38].strip())
        y = float(line[38:46].strip())
        z = float(line[46:54].strip())
        coordinates = [x, y, z]

        if resid in TM1_RESIDS:
            ca_coordinates.setdefault("TM1", []).append(coordinates)
        elif resid in TM2_RESIDS:
            ca_coordinates.setdefault("TM2", []).append(coordinates)
        elif resid in TM3_RESIDS:
            ca_coordinates.setdefault("TM3", []).append(coordinates)
        elif resid in TM4_RESIDS:
            ca_coordinates.setdefault("TM4", []).append(coordinates)
        elif resid in TM5_RESIDS:
            ca_coordinates.setdefault("TM5", []).append(coordinates)
        elif resid in TM6_RESIDS:
            ca_coordinates.setdefault("TM6", []).append(coordinates)
        elif resid in TM7_RESIDS:
            ca_coordinates.setdefault("TM7", []).append(coordinates)

    return ca_coordinates


def get_closest_ca_per_helix(target_coord, ca_coordinates):
    """
    Finds the closest C-alpha coordinate for each helix to a target coordinate.
    Parameters:
        target_coord (Tuple[float, float, float]): The target coordinate (x, y, z).
        ca_coordinates (Dict[str, List[Tuple[float, float, float]]]): A dictionary where keys are helix names
            and values are lists of C-alpha coordinates (x, y, z).
    Returns:
        List[Tuple[float, float, float]]: A list of closest C-alpha coordinates"""
    closest_coords = []
    for helix, coords in ca_coordinates.items():
        closest = min(coords, key=lambda c: ((c[0] - target_coord[0]) ** 2 + (c[1] - target_coord[1]) ** 2 + (c[2] - target_coord[2]) ** 2) ** 0.5)
        closest_coords.append(closest)
    return closest_coords


def calculate_geometric_center(coordinates):
    """
    Calculates the geometric center of a list of 3D coordinates.
    Parameters:
        coordinates (List[Tuple[float, float, float]]): List of 3D coordinates.
    Returns:
        List[float, float, float]: The geometric center (x, y, z).
    """
    x_sum = sum(coord[0] for coord in coordinates)
    y_sum = sum(coord[1] for coord in coordinates)
    z_sum = sum(coord[2] for coord in coordinates)

    n = len(coordinates)
    return [x_sum / n, y_sum / n, z_sum / n]


def calculate_2point_vector(coord1, coord2):
    """
    Calculates the vector between two 3D coordinates.
    Parameters:
        coord1 (Tuple[float, float, float]): The first coordinate (x, y, z).
        coord2 (Tuple[float, float, float]): The second coordinate (x, y, z).
    Returns:
        List[float, float, float]: The vector from coord1 to coord2.
    """
    return [coord2[0] - coord1[0], coord2[1] - coord1[1], coord2[2] - coord1[2]]


def write_pdb_from_coordinates(coordinates_and_betas, output_file):
    """
    Writes a list of (x, y, z) coordinates to a PDB file as ATOM records.
    Parameters:
        coordinates (List[Tuple[float, float, float]]): List of 3D coordinates.
        output_file (str): Path to the output PDB file.
    """
    with open(output_file, 'w') as f:
        for i, (coord, b_factor) in enumerate(coordinates_and_betas):
            x, y, z = coord
            line = (
                f"HETATM{i+1:5d}  C   PTH     1    "
                f"{x:8.3f}{y:8.3f}{z:8.3f}"
                f"{1.00:6.2f}{b_factor:6.4f}           C\n"
            )
            f.write(line)
        f.write("END\n")


In [7]:
# Read the superposition and reference structure files
superposition_coordinates_and_betas = read_pocket_coordinates(superposition)
ca_coordinates = read_ca_coordinates_per_helix(ref_structure)

# Remove TM4 and TM1 from the C-alpha coordinates
# They do not make the core of the protein
del ca_coordinates["TM4"]
del ca_coordinates["TM1"]

In [10]:
keep_cords = []
for (pth_coord, b_factor) in superposition_coordinates_and_betas:
    closest_cas = get_closest_ca_per_helix(pth_coord, ca_coordinates)
    center_coord = calculate_geometric_center(closest_cas)
    
    dot_products = []
    for closest_ca in closest_cas:
        ca2center = calculate_2point_vector(center_coord, closest_ca)
        ca2pth = calculate_2point_vector(pth_coord, closest_ca)
        dot_products.append(np.dot(ca2center[0:2], ca2pth[0:2]))  # Only use x and y components for the dot product

    # Check if all of the dot products are positive
    # If one of them is negative, it means the point is not on the core of
    # the protein, so we keep it
    if all(dp > 0 for dp in dot_products):
        continue
    else:
        keep_cords.append((pth_coord, b_factor))

In [12]:
write_pdb_from_coordinates(keep_cords, "output_grid_filtered.pdb")

In [ ]:
select resn PTH and (b < 0.2);
set sphere_scale, 0.4, sele;
select resn PTH and (b > 0.2 and b < 0.25 or b = 0.2);
set sphere_scale, 0.5, sele;
select resn PTH and (b > 0.25 and b < 0.3 or b = 0.25);
set sphere_scale, 0.6, sele;
select resn PTH and (b > 0.3 and b < 0.35 or b = 0.3);
set sphere_scale, 0.7, sele;
select resn PTH and (b > 0.35 and b < 0.4 or b = 0.35);
set sphere_scale, 0.8, sele;
select resn PTH and (b > 0.4 and b < 0.45 or b = 0.4);
set sphere_scale, 0.9, sele;
select resn PTH and (b > 0.45 and b < 0.5 or b = 0.45);
set sphere_scale, 1, sele;
select resn PTH and (b > 0.5 and b < 0.55 or b = 0.5);
set sphere_scale, 1.1, sele;